Tester et entraîner un modèle de reconnaissance d'écriture
===


Les recueils qui constituent une part importante de la correspondance de Constance de Salm sont l'oeuvre de plusieurs scribes. La différence des écritures représente une difficulté certaine pour la reconnaissance automatique.

# Classer les images par mains

Inventorier toutes les mains attestées dans une source n'est pas toujours aisé.

L'objectif reste avant tout de repérer les mains principales : celles attestées sur le plus grand nombre de pages. Une main qui ne serait attestée que sur une dizaine de page ne mériterais pas d'être classée.

On renseigne alors le fichier [mains.csv](./sources/mains.csv) qui jouera un rôle important dans l'évaluation des modèles et leur entraînement.

# Créer un échantillon-test de chaque écriture

Afin de pouvoir tester le modèle, il est nécessaire de constituer une vérité de terrain de 2-3 doubles pages (selon la densité d'écriture qu'elles contiennent).

On crée dans le dossier de chaque main un dossier **test** contenant des spécimens d'écriture selon les critères suivants :
- Reproductions de bonne qualité (sans problème de transparence)
- Choisies manière discontinue (l'écriture d'une même main peut en effet varier)

Si certaines mains ne sont attestées qu'en compagnie d'autres écritures, on veillera à limiter le test de reconnaissance d'écriture aux seuls lignes de la main à tester (en supprimant après segmentation les lignes non pertinentes afin que la reconnaissance de l'écriture ne les traite pas).

Voici un modèle d'arborescence pour le rangement et le nommage des fichiers et dossiers :
```txt
sources/
├── mainCdS02_Konv002_01/
│   ├── test/
│   │   ├── CdS02_Konv002-02_0065.jpg
│   │   ├── CdS02_Konv002-02_0073.jpg
│   ├── train/
├── mainCdS02_Konv019_01/
│   ├── test/
│   │   ├── CdS02_Konv019_0002.jpg
│   │   ├── CdS02_Konv019_0003.jpg
│   └── train/
├── mains.csv
├── modeles/
└── veriteTerrain/
```


# Tester des modèles HTR

## Installer l'application Kraken

On recommande pour tester un modèle d'utiliser l'application Kraken en ligne de commande, disponible pour Linux et Mac OSX (non pour Windows). Les instructions sont consultables [ici](https://github.com/mittagessen/kraken#installation).

## Importer un modèle

Les modèles extérieurs au projet que l'on a utilisés sont téléchargeables sur le [Gitlab du laboratoire Inria](https://gitlab.inria.fr/dh-projects/kraken-models/-/tree/master/transcription%20models).

## Initier un journal de tests

Le script [journalReconn.py](./py/journalReconn.py) permet de pré-remplir un journal pour l'enregistrement des résultats des tests effectués sur les modèles. 

On lui donne comme argument un nom de modèle et il écrit dans le fichier Json [journal-reconn.json](./sources/journal-reconn.json), à la date et à l'heure courante, pour chaque main listée dans le fichier [mains.csv](./sources/mains.csv), les noms de ces mains et prépare le renseignement des valeurs de test.

Le test initial se fait naturellement sans avoir entraîné le modèle sur les vérités de terrain.

Pour lancer l'écriture du journal sur un modèle particulier :

In [20]:
!python3 py/journalReconn.py --help

Usage: journalReconn.py [OPTIONS] MODELE

  Cette fonction prend comme argument le nom d'un modèle de reconnaissance
  d'écriture, si l'option -v est active, elle analyse les données
  d'entraînement fournies dans le dossier ./sources/veriteTerrain/ puis
  inscrit avec la date courante, les données collectées, dans le fichier
  journal-reconn.json :param modele: nom de modèle HTR :type modele: str
  :return: None

Options:
  -v, --veriteterrain  Prend en compte le contenu du dossier
                       ./sources/veriteTerrain
  --help               Show this message and exit.


Le contenu écrit dans le fichier journal donne comme **accuracy** pour chaque main une valeur de 0. Cette valeur doit être saisie manuellement dans le fichier une fois effectué le test comme suit.


## Effectuer un test

Avec la commande suivante on effectue un test d'acuité pour un modèle sur une main particulière (on doit relever l'*Average accuracy*) :

In [3]:
# ketos test -m ./source/modeles/NOM-MODEL.mlmodel source/NOM-MAIN/test/*xml -f alto
!ketos test -m ./sources/modeles-rec/cm_ft_mrs15_11.mlmodel sources/mainCdS02_Konv002_03/test/*xml -f alto

Loading model ./sources/modeles-rec/cm_ft_mrs15_11.mlmodel	✓
Evaluating ./sources/modeles-rec/cm_ft_mrs15_11.mlmodel
Evaluating ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 72/72 0:00:00 0:00:19 0:00:02 0:00:19
=== report  ===

3766	Characters
682	Errors
81.89%	Accuracy

84	Insertions
163	Deletions
435	Substitutions

Count	Missed	%Right
742	92	87.60%	Common
3024	427	85.88%	Latin

Errors	Correct-Generated
58	{  } - { COMBINING ACUTE ACCENT }
55	{ é } - { e }
29	{ r } - { v }
22	{  } - { SPACE }
21	{  } - { COMBINING GRAVE ACCENT }
20	{ i } - { e }
18	{ à } - { a }
16	{ r } - { s }
15	{ e } - {  }
12	{ è } - { e }
10	{  } - { ^ }
9	{ n } - { r }
9	{  } - { - }
9	{ r } - {  }
8	{ . } - {  }
8	{  } - { s }
7	{ c } - { a }
7	{ u } - { n }
7	{ . } - { SPACE }
7	{ u } - { a }
6	{ ê } - { e }
6	{  } - { t }
6	{ s } - { o }
6	{ a } - { e }
5	{ m } - { n }
5	{ u } - { e }
5	{ r } - { n }
5	{ e } - { i }
5	{ SPACE } - {  }
5	{ r } - { COMBINING GRAVE ACCENT }
5	{  } - { v }
5	{ t } - { l }
4	{ S

Il convient de choisir le modèle présentant les meilleurs résultats pour l'ensemble des écritures testées. Si le meilleur modèle n'atteint pas 90% pour tout ou partie des mains, il convient de l'entraîner par l'apport de vérités de terrain pour chaque main n'atteignant pas ce score.

# Entraîner un modèle

On procède à la constitution d'une vérité de terrain pour chaque spécimen. Pour cela, il n'est pas nécessaire d'annoter les régions d'écriture dans le cadre de la segmentation de la page (comme expliqué dans le notebook [suivant](./2_Realiser_une_prediction_HTR.ipynb)). On se contente d'une segmentation automatique, puis on transcrit les lignes manuellement.